# Tensorflow 

In [3]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import random
import numpy as np

import warnings
warnings.filterwarnings('ignore')

D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

In [ ]:
m_data = fetch_mldata('MNIST original',data_home='./')

X,y= m_data['data'],m_data['target']
x_train,x_test,y_train,y_test = X[:60000],X[60000:],y[:60000],y[60000:]

####  因为有的算法可能会受到训练样例的顺序的影响，当在一行中得到许多相似的样例，算法的效果会很差，所以随机打乱

In [ ]:
random_index = np.random.permutation(60000)

x_train,y_train = x_train[random_index],y_train[random_index]

## 用	TensorFlow	高级	API	训练	MLP


与	__TensorFlow__	一起训练	__MLP__	最简单的方法是使用高级	__API	TF.Learn__ ，这与	__sklearn__	的	__API__ 非常相似。	__DNNClassifier__	可以很容易训练具有任意数量隐层的深度神经网络，而	__softmax__	输出层输出估计的类概率。

例如，下面的代码训练 __两个隐藏层__ 的	__DNN__ （一个具有	__300__	个神经元，另一个具有	__100__	个神经元）和一个具有	__10__	个神经元的	__SOFTMax__	输出层进行分类：


In [ ]:
# feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
# dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100],n_classes=10,feature_columns=feature_columns)
# dnn_clf.fit(x=x_train,y=y_train,batch_size=50,steps=40000)

__DNNClassifier__	基于	__Relu__	激活函数创建所有神经元层（我们可以通过设置超参 数	__activation_fn__	来改变激活函数）。输出层基于	__SoftMax__	函数，__损失函数__ 是 __交叉熵__。


### 使用普通	TensorFlow	训练	DNN


### 一、构造阶段

In [1]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

设置输出特征形状和输出特征形状

In [4]:
X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y = tf.placeholder(tf.int64,shape=(None),name='y')

#### 创建一个图层。它将需要参数来指定输入， 神经元数量，激活函数和图层的名称:


#### 第三行后三行：

创建一个保存权重矩阵的	W	变量。	它将是包含每个输入和每个神经元之间 的所有连接权重的2D张量；因此，它的形状将是	(n_inputs,	n_neurons)	。

它将被随机初始化，__使用具有标准差为	2/√n	的截断的正态（高斯）分布__ (使用 __截断的正态分布而不是常规正态分布确保不会有任何大的权重__ ，这可能会减慢训练。), __使用这个特定的标准差有助于算法的收敛速度更快__。



#### tf.truncated_normal 的用法：https://github.com/OneStepAndTwoSteps/TensorFlow_notes

#### 截断的正态分布 https://www.zhihu.com/question/49923924

In [ ]:
# def neuron_layer(X,n_neurous,name,activation=None):
#     with tf.name_scope(name):
#         n_inputs =int(X.get_shape()[1])
        
#         stddev = 2 /np.sqrt(n_inputs)
#         init = tf.truncated_normal(shape = (n_inputs,n_neurous),stddev=stddev)  # 产生截断的正态分布
#         W = tf.Variable(init,name ='weights')
#         b = tf.Variable(tf.zeros([n_neurous]),name='biases') # 每个神经元 有一个偏置参数 默认为0
#         z = tf.matmul(X,W)+b    # 该向量化实现将有效地计算输入的加权和 加上层中每个神经元的偏置，对于批次中的所有实例，仅需一次. 
#         if activation == 'relu':
#             return tf.nn.relu(z)
#         else:
#             return z

### 构建神经网络

第 一个隐藏层以 X 为输入。	

第二个将第一个隐藏层的输出作为其输入。	

最后，输出层将第二个隐藏层的输出作为其输入。
	

In [ ]:
# with  tf.name_scope('dnn'):
#     hidden1 = neuron_layer(X,n_hidden1,'hidden1',activation='relu')
# hidden2 = neuron_layer(hidden1,n_hidden2,'hidden2',activation='relu')
# logits = neuron_layer(hidden2,n_outputs,'outputs')

请注意，为了清楚起见，我们再次使用名称范围。	

还要注意，logit	是在通过	softmax	激活函 数之前神经网络的输出：为了优化，我们稍后将处理	softmax	计算。


### 使用 tensorflow 内置的功能来创建标准的神经网络层

通常不需要 像我们刚才那样定义你自己的	`neuron_layer()`	函数。	

例如，`TensorFlow` 的	`fully_connected()`	函数创建一个 __完全连接的层__ ，其中所有输入都连接到图层中的所有神经元。	

它使用正确的初始化策略来负责创建 __权重__ 和 __偏置变量__ ，并且 __默认情况__ 下使用	`ReLU`	激活函数（我们可以使用	`activate_fn`	参数来更改它）。	
#### 只需导入该功能，并使用以下代码替换	dnn	构建部分：
	

In [ ]:
# from tensorflow.contrib.layers import fully_connected

In [ ]:
# with tf.name_scope('dnn'):
#     hidden1 = fully_connected(X,n_hidden1,scope='hidden1')
#     hidden2 = fully_connected(hidden1,n_hidden2,scope='hidden2')
#     logits = fully_connected(hidden2,n_outputs,scope='outputs',activation_fn=None)

`tensorflow.contrib`	包包含许多有用的功能，但它是一个尚未分级成为主要	`TensorFlow	API` 一部分的实验代码的地方。因此,`full_connected()`	函数（和任何其他 `contrib` 代码）可能会在将来更改或移动。

### 所以：__使用 `dense()` 代替 `neuron_layer()`__

__dense()__	函数与	__fully_connected()__	函数几乎相同，除了一些细微的差别：

* __scope__	变为名称


* __activation_fn__	变为激活（同样	\_fn	后缀从其他参数 （如	__normalizer_fn__	）中删除）


* __weights_initializer__	成为	__kernel_initializer__	等。


* 默认激活现在是无，而不是	__tf.nn.relu__	。

In [5]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X,n_hidden1,name='hidden1',activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1,n_hidden2,name='hidden2',activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2,n_outputs,name='outputs')

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


### 训练损失函数

In [6]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')

有了 __神经网络__ 和 __损失函数__ 之后，接下来定义一个 __梯度下降优化器__

In [7]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

建模阶段的最后一个重要步骤是指定如何 __评估模型__。	我们将简单地将精度用作我们的绩效指标

首先，对于每个实例，通过检查最高	logit	是否对应于目标类别来确定神经网络的预测是 否正确。	为此，您可以使用	in_top_k()	函数。	这返回一个充满布尔值的	1D	张量，因此我们 需要将这些布尔值转换为浮点数，然后计算平均值。	这将给我们网络的整体准确性.


In [8]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

接下来 创建一个初始化所有变量的节点，我们还将创建一个	Saver	来 将我们训练有素的模型参数保存到磁盘中：


In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### 二、执行阶段

使用 __tensorflow__ 自己提供的工具来加载 __mnist__ 数据集

TensorFlow	提供了自己的助手来获取数据，将其缩放（0	到	1	之间），将 它洗牌，并提供一个简单的功能来一次加载一个小批量：


值得注意的是：__read_data_sets__ 是用于读取压缩包的

#### tensorflow.examples.tutorials 现在已经弃用了，推荐使用tensorflow.keras.datasets 

In [18]:
from tensorflow.examples.tutorials.mnist import input_data
# from keras.datasets import mnist

In [19]:
mnist = input_data.read_data_sets('./mldata/')

# (x_train,y_train),(x_test,y_test) = mnist.load_data('./mnist/mnist.npz')

Extracting ./mldata/train-images-idx3-ubyte.gz
Extracting ./mldata/train-labels-idx1-ubyte.gz
Extracting ./mldata/t10k-images-idx3-ubyte.gz
Extracting ./mldata/t10k-labels-idx1-ubyte.gz


定义 __迭代数__ 和 __小批量的大小__

In [30]:
n_epochs = 21
batch_size = 500

#### 现在我们去训练模型:

In [31]:
with tf.Session() as sess:
    init.run()    # 初始化之前定义的init
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch  = mnist.train.next_batch(batch_size)       #其中的n代表返回多少个 训练数据集 和 对应的标签
            sess.run(training_op,feed_dict={X:x_batch,y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:x_batch,y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        
        print(epoch,'Train accuracy: ',acc_train,'Test accuracy',acc_test)
    save_path = saver.save(sess,'./my_model_final.ckpt')

0 Train accuracy:  0.618 Test accuracy 0.6583
1 Train accuracy:  0.8 Test accuracy 0.8137
2 Train accuracy:  0.84 Test accuracy 0.8507
3 Train accuracy:  0.874 Test accuracy 0.8696
4 Train accuracy:  0.87 Test accuracy 0.88
5 Train accuracy:  0.86 Test accuracy 0.8893
6 Train accuracy:  0.882 Test accuracy 0.8962
7 Train accuracy:  0.872 Test accuracy 0.9011
8 Train accuracy:  0.908 Test accuracy 0.9043
9 Train accuracy:  0.908 Test accuracy 0.9076
10 Train accuracy:  0.89 Test accuracy 0.9104
11 Train accuracy:  0.908 Test accuracy 0.9111
12 Train accuracy:  0.89 Test accuracy 0.9136
13 Train accuracy:  0.922 Test accuracy 0.9155
14 Train accuracy:  0.882 Test accuracy 0.9164
15 Train accuracy:  0.906 Test accuracy 0.9178
16 Train accuracy:  0.906 Test accuracy 0.9199
17 Train accuracy:  0.906 Test accuracy 0.9198
18 Train accuracy:  0.894 Test accuracy 0.9202
19 Train accuracy:  0.936 Test accuracy 0.9222
20 Train accuracy:  0.914 Test accuracy 0.9231


接下来，在每个时期结束时，代码评估最后一个小批量和完整训练集上的模型， 并打印出结果。	最后，模型参数保存到磁盘。


###  三、使用神经网络

In [36]:
with tf.Session() as sess:
    saver.restore(sess,'./my_model_final.ckpt')
    X_train = mnist.test.images[:50]
    Z = logits.eval(feed_dict={X:X_train})
    y_pred = np.argmax(Z,axis=1)
    print(y_pred)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 6 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4]


In [40]:
Z[1]

array([ 1.7259476, -2.2674694,  6.025392 ,  2.3775709, -8.074056 ,
        1.3455541,  3.325245 , -6.7976556,  0.4249972, -7.7564635],
      dtype=float32)

In [38]:
mnist.test.labels[:50]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4], dtype=uint8)

首先，代码从磁盘加载模型参数。	然后加载一些您想要分类的新图像。	记住应用与训练数据 相同的特征缩放（在这种情况下，将其从	0	缩放到	1）。	然后代码评估对数点节点。	

如果您 想知道所有估计的类概率，则需要将	softmax()	函数应用于对数，但如果您只想预测一个 类，则可以简单地选择具有最高	logit	值的类（使用	argmax()	函数做的伎俩）。


## 以上建模和训练流程：

1、输入和目标创建占位符

2、创建了一个构建神经元层的函数，我们用它来创建 DNN

3、我们定义了损失函数

4、我们创建了一 个优化器

5、然后定义了性能指标。	

6、最后到执行阶段。
